- 지금까지는 파이썬 내장 객체에 대한 구조와 동작에 대해 배웠다면, 이제는 사용자 정의 클래스를 이용해 원하는 동작을 구현해본다
- 다양한 특별메서드와 @classmethod, @staticmethod 데코레이터의 사용법도 배운다

# 특별메서드 활용

In [46]:
from array import array
import math

class Vector2d:
    typecode = 'd'

    def __init__(self, x, y) -> None:
        self.__x = float(x)
        self.__y = float(y)

    @property
    def x(self):
        return self.__x

    @property
    def y(self):
        return self.__y

    def __iter__(self):
        '''@property 데코레이터를 통해 self.x, self.y를 반환하므로 내부 객체가 변경될 걱정이 없다'''
        return (i for i in (self.x, self.y)) 
    
    def __repr__(self) -> str:
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)
    
    def __str__(self) -> str:
        return str(tuple(self))
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) + bytes(array(self.typecode, self)))

    def __eq__(self, __o: object) -> bool:
        return tuple(self) == tuple(__o)

    def __abs__(self):
        return math.hypot(self.x, self.y)
    
    def __bool__(self):
        return bool(abs(self))

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(*memv)

    def __format__(self, __format_spec: str) -> str:
        '''p 포메터를 통해 극좌표를 반환하도록 만든다'''
        if __format_spec.endswith('p'):
            __format_spec = __format_spec[:-1]
            coords = (abs(self), self.angle())
            outer_format = '<{}, {}>'
        else:
            coords = self
            outer_format = '({}, {})'
        components = (format(c, __format_spec) for c in coords)
        return outer_format.format(*components)

    def angle(self):
        return math.atan2(self.y, self.x)

    def __hash__(self) -> int:
        return hash(self.x) ^ hash(self.y)

In [17]:
v1 = Vector2d(3, 4)
print(v1.x, v1.y)

3.0 4.0


In [18]:
x, y = v1
x, y

(3.0, 4.0)

In [19]:
v1

Vector2d(3.0, 4.0)

In [20]:
v1_clone = eval(repr(v1))
v1 == v1_clone

True

In [21]:
print(v1)

(3.0, 4.0)


In [22]:
octets = bytes(v1)
octets

b'd\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\x10@'

In [23]:
abs(v1)

5.0

In [24]:
bool(v1), bool(Vector2d(0, 0))

(True, False)

In [1]:
class Demo:
		@classmethod
		def classmeth(*args):
				return args
		@staticmethod
		def statmeth(*args):
				return args

In [2]:
Demo.classmeth()

(__main__.Demo,)

In [3]:
Demo.classmeth('spam')

(__main__.Demo, 'spam')

In [4]:
Demo.statmeth()

()

In [5]:
Demo.statmeth('spam')

('spam',)

- classmethod는 객체지향에서 우리가 흔히 알고있는 staticmethod와 비슷하게 동작한다. 첫번째 매개변수로 class 자기자신을 받아서 생성자를 호출한다. 
- 반면 staticmethod는 그냥 일반 함수와 완전히 같다. self, cls 와 같은 클래스 속성을 전혀 받지 않고 실행할 수 있다.(이 책에서 필자는 staticmethod가 왜 필요한지 모르겠다고 까지 말한다)

In [26]:
v1 = Vector2d(3, 4)
octets = bytes(v1)
v1_octets = v1.frombytes(octets)
v1_octets == v1

True

In [27]:
v1 = Vector2d(3, 4)
format(v1)

'(3.0, 4.0)'

In [28]:
format(v1, '.2f')

'(3.00, 4.00)'

In [40]:
format(v1, '010.2f')

'(0000003.00, 0000004.00)'

In [43]:
format(Vector2d(1, 1), 'p')

'<1.4142135623730951, 0.7853981633974483>'

In [44]:
format(Vector2d(1, 1), '.3ep')

'<1.414e+00, 7.854e-01>'

In [45]:
format(Vector2d(1, 1), '0.5fp')

'<1.41421, 0.78540>'

In [47]:
v1 = Vector2d(3, 4)
v2 = Vector2d(3.1, 4.2)
hash(v1), hash(v2)

(7, 384307168202284039)

In [48]:
set([v1, v2])

{Vector2d(3.0, 4.0), Vector2d(3.1, 4.2)}

- 어플리케이션에서 사용하지 않을 특별메서드까지 구현할 필요는 없다.
- 유동적으로 필요한것만 하면 꽤 파이썬스러운 객체를 만들 수 있다.
- 사용자는 객체가 파이썬스러운지 신경쓰지 않는다.

# private, protected 속성
- 변수앞에 __를 붙이는 것으로 private를, _를 붙이는 것으로 protected를 표시한다고 하지만, 실제로 접근을 막을 수 있는 방법은 없다.
- 어디까지나 보호의 차원이고, 모든 파이썬 개발자가 명목상 지켜야 할 약속인 것이다.

# `__slot__` 클래스 속성으로 공간 절약
- 파이썬 객체는 변수 및 메서드를 모두 딕셔너리에 저장한다.
- 딕셔너리 자체는 빠른속도를 위해 공간을 희생한 자료구조로써 변수는 몇 개 없지만 수백만개의 객체가 필요한 경우 오버헤드가 크다.
- 그래서 변수를 튜플에 저장하는 방법을 통해 메모리 사용량을 극적으로 줄일 수 있다.

In [49]:
class Vector2d:
    __slots__ = ('__x', '__y')

    typecode = 'd'

    def __init__(self, x, y) -> None:
        self.__x = float(x)
        self.__y = float(y)

    @property
    def x(self):
        return self.__x

    @property
    def y(self):
        return self.__y

    def __iter__(self):
        '''@property 데코레이터를 통해 self.x, self.y를 반환하므로 내부 객체가 변경될 걱정이 없다'''
        return (i for i in (self.x, self.y)) 
    
    def __repr__(self) -> str:
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)
    
    def __str__(self) -> str:
        return str(tuple(self))
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)]) + bytes(array(self.typecode, self)))

    def __eq__(self, __o: object) -> bool:
        return tuple(self) == tuple(__o)

    def __abs__(self):
        return math.hypot(self.x, self.y)
    
    def __bool__(self):
        return bool(abs(self))

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(*memv)

    def __format__(self, __format_spec: str) -> str:
        '''p 포메터를 통해 극좌표를 반환하도록 만든다'''
        if __format_spec.endswith('p'):
            __format_spec = __format_spec[:-1]
            coords = (abs(self), self.angle())
            outer_format = '<{}, {}>'
        else:
            coords = self
            outer_format = '({}, {})'
        components = (format(c, __format_spec) for c in coords)
        return outer_format.format(*components)

    def angle(self):
        return math.atan2(self.y, self.x)

    def __hash__(self) -> int:
        return hash(self.x) ^ hash(self.y)